In [1]:
import sys
print(sys.executable)

sys.path.append('../')

/home/mshunya/project/100_GQCO/work/2024102201_PerfomanceEval/.env/bin/python


In [2]:
import pickle
import time
import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Batch
from IPython.display import display, Math
from gqco.train import MyModel
from gqco.utils import fix_seed, arange_token
from gqco.data import generate_data, RandomGraphDatasetWithClone
from gqco.solve import solve_from_token, plot_from_dict, brute_solver, probs_to_result
from gqco.model import TransformerWithMoE
from gqco.quantum import coef_to_pauli, pauli_to_matrix, make_cudaq_operator
from gqco.task import GQCO

import importlib
def fun_reload(pkg):
    module = importlib.import_module(pkg)
    module = importlib.reload(module)
    attrs = {attr: getattr(module, attr) for attr in dir(module)}
    globals().update(attrs)

fun_reload('gqco.quantum')
fun_reload('gqco.model')

In [3]:
def data_from_adj(adj, args, num_clone, device):
    dataset = RandomGraphDatasetWithClone(adj, num_clone=num_clone, max_num_nodes=args.max_size, device=device)
    record = Batch.from_data_list(dataset)
    record['size'] = record['size'].tolist()[0]
    record['len'] = dataset.len()

    return adj, size, record


def _get_answer(dct, metric='min'):
    vals = list(dct.values())
    if metric=='min':
        target_val = min(vals)
    if metric=='max':
        target_val = max(vals)
    target_keys = [key for key, value in dct.items() if value == target_val]
    return target_keys, target_val

In [4]:
task_path = '../model/taskobjects.pkl'
checkpoint_path = '../model/latest-current_size=10-epoch=2999.ckpt'
testdata_path = '../work/testdata.pkl'
seed=0

In [5]:
with open(task_path, 'rb') as f: 
    obj = pickle.load(f)
# task = obj['task']
args = obj['args']
# args.quantum_tool='cudaq'
task = GQCO(args)

In [6]:
model = TransformerWithMoE(args)
model = MyModel.load_from_checkpoint(checkpoint_path, model=model, task=task, args=args)

In [7]:
with open(testdata_path, 'rb') as f:
    testdata = pickle.load(f)

In [8]:
size_list = [s for s in range(3, 11)]

## Setup

In [9]:
SEED = 373
num_clone = 100
temperature = 2.0
device = model.device

size_list = [s for s in range(3, 11)]
solver_type = ['brute']

## Brute solver

In [66]:
ans = {}
tms = {}

for size in size_list:
    _ans = []
    _tms = []
    
    for adj in tqdm.tqdm(testdata[size], desc=f'size: {size}'):
        adj, size, record = data_from_adj(adj, args, num_clone, device)

        _s = time.time()
        dict_true = brute_solver(adj)
        _e = time.time()

        min_keys_true, min_val_true = _get_answer(dict_true, metric='min')

        _ans.append(min_keys_true)
        _tms.append(_e - _s)

    ans[size] = _ans
    tms[size] = _tms

with open('./trueans.pkl', 'wb') as f:
    pickle.dump({'answer': ans, 'time': tms}, f)

size: 10: 100%|█████████████████████████████████████████████████████████████████████| 1000/1000 [05:14<00:00,  3.18it/s]


## GQCO solver

In [24]:
task.tool = 'cudaq'
# task.tool = 'qiskit'

In [57]:
# t_list = [1.0, 2.0]
# cl_list = [1, 5, 10, 20, 100]
t_list = [1.0, 2.0]
cl_list = [20]

In [ ]:
model.eval()

result = []

for temperature in t_list:
    for num_clone in cl_list:
        ans = {}
        tms = {}
        for size in size_list:
            _ans = []
            _tms = []
        
            count = 0
            for adj in tqdm.tqdm(testdata[size], desc=f'temperature: {temperature}, clone: {num_clone}, size: {size}'):
                count += 1
                adj, size, record = data_from_adj(adj.to(device), args, num_clone, device)
                fix_seed(SEED)
        
                _s = time.time()
                with torch.no_grad():
                    out_tokens, probs_all, _, logits_all = model.model.forward(record, temperature=temperature, same_token_penalty=0.0, masked_tokens=task.bad_tokens[size], deterministic=False) 
                tokens_list = [arange_token(t, args) for t in out_tokens.detach().cpu().tolist()]
                _t1 = time.time()
        
                energies = []
                vectors = []
                energy_cache = {}
                vector_cache = {}
                for t in tokens_list:
                    t_tuple = tuple(t)
                    if t_tuple not in energy_cache:
                        energy = task.compute_energy(t, adj, args.num_shot)
                        energy_cache[t_tuple] = energy
                    else:
                        energy = energy_cache[t_tuple]
                    energies.append(energy)
        
                ## Get the best
                idx_min = energies.index(min(energies))
                token_min = tokens_list[idx_min]
                energy_min = energies[idx_min]
            
                while token_min and token_min[-1] == 0:
                    token_min.pop()
                _t2 = time.time()
        
                qc = task.get_circuit(token_min, size=len(adj))
                vector = qc.get_state()
                probs = np.abs(vector)**2
                _t3 = time.time()
        
                dict_pred = probs_to_result(probs)
                min_keys, min_val = _get_answer(dict_pred, metric='max')
        
                _ans.append(min_keys)
                _tms.append([_s, _t1, _t2, _t3])
        
                if count == 100:
                    break
                
            ans[size] = _ans
            tms[size] = _tms
        result.append({
            'temperature': temperature,
            'num_clone': num_clone,
            'answers': ans,
            'times': tms
        })


    with open('./gqcoans.pkl', 'wb') as f:
        pickle.dump(result, f)

temperature: 2.0, clone: 20, size: 8:   0%|▏                                           | 5/1000 [00:04<15:39,  1.06it/s]

In [103]:
with open('../work/outputs/trueans.pkl', 'rb') as f:
    dict_true = pickle.load(f)
# with open('./gqcoans.pkl', 'rb') as f:
#     dict_gqco = pickle.load(f)

In [6]:
dict_gqco[0]['temperature']

1.0

In [54]:
_tmp = []
_clo = []
_siz = []
_acc = []
_tim = []
_tim1 = []
_tim2 = []
for size in size_list:

    _tmp.append(None)
    _clo.append(None)
    _siz.append(size)
    _acc.append(None
    _tim.append(sum(dict_true['time'][size])/len(dict_true['time'][size]))

    ## GQ
    for temperature in t_list:
        for num_clone in cl_list:
            is_correct = 0
            sum_truetime = 0
            sum_gqcotime1 = 0
            sum_gqcotime2 = 0
            sum_gqcotime_all = 0
            # for trueans, gqcoans in zip(dict_true['answer'][size], dict_gqco['answer'][size]):
            for i in range(min(len(dict_true['answer'][size]), len(dict_gqco['answer'][size]))):
                trueans = dict_true['answer'][size][i]
                gqcoans = dict_gqco['answer'][size][i]

                truetime = dict_true['time'][size][i]
                gqcotime = dict_gqco['time'][size][i]
                
                if len(set(gqcoans) & set(trueans)):
                    is_correct += 1

            _tmp.append(temperature)
            _clo.append(num_clone)
            _siz.append(size)
            _acc.append(is_correct / (i+1))
            _tim.append()

## QAOA solver

In [9]:
import numpy as np
import cudaq
from cudaq import spin
from typing import List


In [78]:
def adj_to_cudaqlist(adj):
    lst1 = []
    lst2 = []
    
    for i in range(len(adj)):
        lst1.append(adj[i, i].cpu().tolist())
    for i in range(len(adj)-1):
        for j in range(i+1, len(adj)):
            lst2.append(adj[i, j].cpu().tolist())

    return lst1, lst2


@cudaq.kernel
def kernel_qaoa(qubit_count: int, layer_count: int, lst1: List[float],
                lst2: List[float], thetas: List[float]):

    qreg = cudaq.qvector(qubit_count)

    h(qreg)

    for ell in range(layer_count):

        for i in range(qubit_count):
            w = lst1[i]
            rz(2.0 * thetas[ell] * w, qreg[i])

        count = 0
        for i in range(qubit_count-1):
            for j in range(i+1, qubit_count):
                w = lst2[count]
                x.ctrl(qreg[i], qreg[j])
                rz(2.0 * thetas[ell] * w, qreg[j])
                x.ctrl(qreg[i], qreg[j])
                count += 1
            
        # Add the mixer kernel to each layer
        for k in range(qubit_count):
            rx(2.0 * thetas[ell + layer_count], qreg[k])


In [ ]:
cudaq.set_target('nvidia')

In [136]:
size = 3
smpl += 1
adj = testdata[size][smpl]


lst1, lst2 = adj_to_cudaqlist(adj)

spin_operator = make_cudaq_operator(adj)

qubit_count = len(adj)
layer_count = 2
parameter_count = 2 * layer_count
SEED = 373

# Specify the optimizer and its initial parameters.
cudaq.set_random_seed(SEED)
optimizer = cudaq.optimizers.NelderMead()
np.random.seed(SEED)
optimizer.initial_parameters = np.random.uniform(-np.pi / 8, np.pi / 8, parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

def objective(parameters):
    return cudaq.observe(kernel_qaoa, spin_operator, qubit_count, layer_count,
                         lst1, lst2, parameters).expectation()

optimal_expectation, optimal_parameters = optimizer.optimize(dimensions=parameter_count, function=objective)

np.array(cudaq.get_state(kernel_qaoa, qubit_count, layer_count, lst1, lst2, optimal_parameters))  ## the output is wrong without this line (why?).
vector = np.array(cudaq.get_state(kernel_qaoa, qubit_count, layer_count, lst1, lst2, optimal_parameters))  ## the output is wrong without this line (why?).
probs = np.abs(vector)**2

result = {}
nqubit = int(np.log2(len(probs)))
    
for i, b in enumerate(range(2**nqubit)):
    # bit = f'{b:0>{nqubit}b}'
    bit = f'{b:0>{nqubit}b}'[::-1]
    result[bit] = probs[i]

result

Initial parameters =  [0.15262786126672012, 0.06538864442917519, 0.2021024886981121, -0.27998268286945754]
optimal_expectation = -1.9098221518466407
Therefore, the max cut value is at least  1.9098221518466407
optimal_parameters = [0.8973663688221337, 1.3963265287738276, 0.8053198105776708, -0.8004052749686027]


{'000': 0.005370893,
 '100': 0.6261513,
 '010': 0.06928239,
 '110': 0.15378143,
 '001': 0.00836413,
 '101': 0.09574778,
 '011': 0.022322103,
 '111': 0.01897973}

In [139]:
min_keys, min_val = _get_answer(result, metric='max')

In [140]:
min_keys

['100']

In [137]:
dict_true['answer'][3][smpl]

['100']

## Make result csv

In [4]:
import pandas as pd

In [9]:
with open('../work/outputs/trueans.pkl', 'rb') as f:
    dict_true = pickle.load(f)

_s = []
_t = []
for size in size_list:
    _s += [size]
    _t += [sum(dict_true['time'][size])/len(dict_true['time'][size])]

pd.DataFrame({'size': _s, 'time': _t}).to_csv('../work/Brute.csv', index=False)

In [10]:
# t_list = [1.0, 2.0]
# cl_list = [1, 5, 10, 20, 100]
sd_list = [0, 373, 42]
t_list = [1.0, 2.0]
cl_list = [1, 5, 10, 20, 100]

_s = []
_temp = []
_cl = []
_t = []
_t1 = []
_t2 = []
_t3 = []
_acc = []
for temp in t_list:
    for cl in cl_list:
        for size in size_list:
            with open(f'../work/outputs/gqcoans_t{temp}_cl{int(cl)}_s{size}.pkl', 'rb') as f:
                dct = pickle.load(f)

            # is_correct = 0
            sum_t = 0
            sum_t1 = 0
            sum_t2 = 0
            # sum_t3 = 0
            for i in range(len(dct['answer'])):
                gqcoans = dct['answer'][i]
                trueans = dict_true['answer'][size][i]

            #     if len(set(gqcoans) & set(trueans)):
            #         is_correct += 1

                sum_t += dct['time'][i][2] - dct['time'][i][0]
                sum_t1 += dct['time'][i][1] - dct['time'][i][0]
                sum_t2 += dct['time'][i][2] - dct['time'][i][1]
            #     sum_t3 += dct['time'][i][3] - dct['time'][i][2]

            _s += [size]
            _temp += [temp]
            _cl += [cl]
            _t += [sum_t / len(dct['answer'])]
            _t1 += [sum_t1 / len(dct['answer'])]
            _t2 += [sum_t2 / len(dct['answer'])]
            # _t3 += [sum_t3 / len(dct['answer'])]
            _acc += [sum(dct['answer']) / len(dct['answer'])]

pd.DataFrame({
    'size': _s, 
    'temperature': _temp,
    'num_clone': _cl,
    'time': _t,
    'time1': _t1,
    'time2': _t2,
    # 'time3': _t3,
    'accuracy': _acc
}).to_csv('../work/GQCO.csv', index=False)

NameError: name 'dict_true' is not defined

In [47]:
r_list = [10]
sw_list = [100, 1000, 10000, 100000, 1000000]

_s = []
_r = []
_sw = []
_t = []
_acc = []
for r in r_list:
    for sw in sw_list:
        for size in size_list:
            with open(f'../work/outputs/saans_r{int(r)}_sw{int(sw)}_s{size}.pkl', 'rb') as f:
                dct = pickle.load(f)

            is_correct = 0
            sum_t = 0
            for i in range(len(dct['answer'])):
                saans = dct['answer'][i]
                trueans = dict_true['answer'][size][i]

                if len(set(saans) & set(trueans)):
                    is_correct += 1

                sum_t += dct['time'][i]

            _s += [size]
            _r += [r]
            _sw += [sw]
            _t += [sum_t / len(dct['answer'])]
            _acc += [is_correct / len(dct['answer'])]

pd.DataFrame({
    'size': _s, 
    'num_reads': _r,
    'num_sweeps': _sw,
    'time': _t,
    'accuracy': _acc
}).to_csv('../work/SA.csv', index=False)

In [21]:
# l_list = [2, 5, 10, 20]
l_list = [1, 2, 3, 5]

_s = []
_l = []
_t = []
_acc = []
for l in l_list:
    for size in size_list:
        with open(f'../work/outputs/qaoaans_l{int(l)}_s{size}.pkl', 'rb') as f:
            dct = pickle.load(f)

        is_correct = 0
        sum_t = 0
        for i in range(len(dct['answer'])):
            qaoaans = dct['answer'][i]
            trueans = dict_true['answer'][size][i]

            if len(set(qaoaans) & set(trueans)):
                is_correct += 1

            sum_t += dct['time'][i]

        _s += [size]
        _l += [l]
        _t += [sum_t / len(dct['answer'])]
        _acc += [is_correct / len(dct['answer'])]

pd.DataFrame({
    'size': _s, 
    'num_repeats': _l,
    'time': _t,
    'accuracy': _acc
}).to_csv('../work/QAOA.csv', index=False)

In [ ]:
with open(f'./outputs/gqcoans_t{temperature}_cl{num_clone}_s{size}.pkl', 'wb') as f:
    pickle.dump({
        'answer': _ans,
        'time': _tms
    }, f)

In [ ]:
                if len(set(gqcoans) & set(trueans)):
                    is_correct += 1

In [147]:
_s

[3, 4, 5, 6, 7, 8, 9, 10]

In [148]:
_t

[0.00027885723114013674,
 0.000881983757019043,
 0.002546091318130493,
 0.00713453459739685,
 0.018621047496795655,
 0.048564555644989016,
 0.12227458953857422,
 0.29894775700569154]

In [145]:
len(dict_true['time'][size])

1000

In [96]:
probs

array([7.2367787e-01, 1.5339936e-01, 5.8508940e-02, 1.4820998e-02,
       6.5545803e-03, 1.1745478e-02, 1.4748685e-04, 3.1145241e-02],
      dtype=float32)

In [ ]:
job = execute(qc, backend=Aer.get_backend('statevector_simulator'))
vector = job.result().get_statevector()
array = []
for v in vector:
    array.append(v)
probs = np.abs(np.array(array))**2

result = {}
nqubit = int(np.log2(len(probs)))
    
for i, b in enumerate(range(2**nqubit)):
    bit = f'{b:0>{nqubit}b}'
    # bit = f'{b:0>{nqubit}b}'[::-1]
    result[bit] = probs[i]

In [85]:
cudaq.observe(kernel_qaoa2, spin_operator, qubit_count, layer_count,
                         lst1, lst2, optimizer.initial_parameters).expectation()

0.0

In [86]:
lst1

[-1.0, -0.954227089881897, -0.8873337507247925]

In [87]:
lst2

[-0.7956457734107971, -0.43199869990348816, -0.05725953355431557]

In [20]:
energy = cudaq.observe(self.kernel, spin_operator, shots_count=num_shot).expectation()

In [74]:
import cudaq
cudaq.set_target('qpp-cpu')


# Define our kernel.
@cudaq.kernel
def kernel(qubit_count: int):
    # Allocate our qubits.
    qvector = cudaq.qvector(qubit_count)
    # Place the first qubit in the superposition state.
    h(qvector[0])
    # Loop through the allocated qubits and apply controlled-X,
    # or CNOT, operations between them.
    for qubit in range(qubit_count - 1):
        x.ctrl(qvector[qubit], qvector[qubit + 1])
    # Measure the qubits.
    mz(qvector)

In [75]:
qubit_count = 2
print(cudaq.draw(kernel, qubit_count))
results = cudaq.sample(kernel, qubit_count)
# Should see a roughly 50/50 distribution between the |00> and
# |11> states. Example: {00: 505  11: 495}
print("Measurement distribution:" + str(results))

RuntimeError: qpp::applyCTRL(): Subsystems mismatch dimensions! [ctrl/dims]

In [55]:
acc

{10: 0.92}

In [47]:
num_clone

100

In [56]:
_tms

[[1729576835.8263264,
  1729576836.3530612,
  1729576839.9159443,
  1729576839.9160333]]

In [133]:
tms[8][0][1] - tms[8][0][0]

0.18765616416931152

In [141]:
tms[8][0][2] - tms[8][0][1]

0.3630859851837158

In [140]:
tms[8][0][3] - tms[8][0][2]

9.226799011230469e-05

In [144]:
tokens_list[0]

[301,
 61,
 61,
 61,
 931,
 1313,
 281,
 552,
 931,
 382,
 1124,
 931,
 1540,
 1578,
 1638,
 951,
 1524,
 1314]

In [145]:
probs_all

tensor([[[0.0000e+00, 5.8094e-05, 2.8459e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.6523e-05, 7.8871e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.3991e-05, 5.8746e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [8.5405e-04, 2.1273e-04, 1.6031e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.5707e-04, 2.8691e-04, 2.0351e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.7887e-04, 1.1520e-04, 1.7305e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 5.8094e-05, 2.8459e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 9.0451e-05, 6.5991e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 3.3207e-05, 1.7845e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [3.0853e-04, 5.8113e-04, 1.5461e-03,  ..., 0.0000e+00,
          0.000

In [148]:
from torch.nn import functional as F

In [151]:
for w in range(len(tokens_list)):
    token_w = tokens_list[w]
    probs_w = probs_all[w, :len(token_w)].clone()
    onehot_w = F.one_hot(torch.tensor(token_w), num_classes=args.vocab_size).to(device)
    log_psum_w = torch.sum(torch.log(probs_w**onehot_w) / len(token_w))
    print(log_psum_w)

tensor(-1.4553, device='cuda:0')
tensor(-3.0167, device='cuda:0')
tensor(-2.4053, device='cuda:0')
tensor(-1.5233, device='cuda:0')
tensor(-1.5086, device='cuda:0')
tensor(-1.8346, device='cuda:0')
tensor(-1.2233, device='cuda:0')
tensor(-4.2374, device='cuda:0')
tensor(-2.2554, device='cuda:0')
tensor(-2.3396, device='cuda:0')
tensor(-0.2694, device='cuda:0')
tensor(-1.3898, device='cuda:0')
tensor(-1.9616, device='cuda:0')
tensor(-1.6924, device='cuda:0')
tensor(-0.9581, device='cuda:0')
tensor(-1.1433, device='cuda:0')
tensor(-1.6621, device='cuda:0')
tensor(-1.4130, device='cuda:0')
tensor(-0.8724, device='cuda:0')
tensor(-3.8516, device='cuda:0')


In [150]:
log_psum_w

tensor(-1.4553, device='cuda:0')

In [142]:
probs_all

tensor([[[0.0000e+00, 5.8094e-05, 2.8459e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.6523e-05, 7.8871e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.3991e-05, 5.8746e-05,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [8.5405e-04, 2.1273e-04, 1.6031e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [8.5707e-04, 2.8691e-04, 2.0351e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [9.7887e-04, 1.1520e-04, 1.7305e-03,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 5.8094e-05, 2.8459e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 9.0451e-05, 6.5991e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 3.3207e-05, 1.7845e-04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [3.0853e-04, 5.8113e-04, 1.5461e-03,  ..., 0.0000e+00,
          0.000

In [5]:
import random
import pickle
import numpy as np
import seaborn as sns
import torch
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import display, Math
from gqco.train import MyModel
from gqco.model import CustomEncoderDecoderModel
from gqco.utils import fix_seed
from gqco.train import generate_data, _arange_token
from gqco.solve import solve_from_token, plot_from_dict, adj_to_text

import importlib
def fun_reload(pkg):
    module = importlib.import_module(pkg)
    module = importlib.reload(module)
    attrs = {attr: getattr(module, attr) for attr in dir(module)}
    globals().update(attrs)

fun_reload('gqeco.solve')

ModuleNotFoundError: No module named 'gqco.train_lightning'

In [3]:
def mysetup(_type):

    if _type == 'only3':
        job_id = 42719343
        epoch = 26799
        num_qubit = 3
        size_list = [3]
    if _type == '3to5':        
        job_id = 42745012
        epoch = 27999
        num_qubit = 5
        size_list = [3, 4, 5]
    
    with open(f'outputs/{job_id}/taskobjects.pkl', 'rb') as f: 
        obj = pickle.load(f)
    gqeco = obj['task']
    args = obj['args']

    model = CustomEncoderDecoderModel(args)
    model = MyModel.load_from_checkpoint(f'outputs/{job_id}/checkpoints/epoch={epoch}.ckpt', model=model, task=gqeco, args=args)

    return model, gqeco, args, num_qubit, size_list

In [4]:
def Mysolver(taskobj, model, adj, record, args, num_try=1, temperature=1, same_token_penalty=0.0, num_shot=-1, is_print=True, random_token=False, cast=False, savefile=None):

    nq = max(torch.count_nonzero(abs(adj).sum(0)), torch.count_nonzero(abs(adj).sum(1)))
    # print(nq)

    if is_print:
        print('-- Objective --')
        adj_to_text(adj)
        print('\n')
    
    if not random_token:
        with torch.no_grad():  
            if cast:
                with torch.autocast('cpu'):
                    out_tokens, probs_all, _, logits_all = model.model.forward(record, temperature=temperature)
            else:
                out_tokens, probs_all, _, logits_all = model.model.forward(record, temperature=temperature)

        tokens_list = [_arange_token(t, args) for t in out_tokens.detach().tolist()]
    if random_token:
        tokens_list = []
        for i in range(num_try):
            _len = random.randint(args.min_generation, args.max_generation)
            t = [random.randint(1, args.vocab_size-1) for _ in range(_len)]
            tokens_list += [t]

    es = [taskobj.compute_energy(t, adj, num_shot) for t in tokens_list]
    index_of_min = es.index(min(es))
    min_token = tokens_list[index_of_min]
    min_energy = es[index_of_min]

    dict_pred, dict_true, qc = solve_from_token(taskobj, min_token, adj, is_print, savefile)

    if is_print:
        print('-- Result --')
        plot_from_dict(dict_pred, dict_true, savefile)


    model_ans = max(dict_pred, key=dict_pred.get)
    exact_ans = min(dict_true, key=dict_true.get)


    return {
        'model token': min_token, 
        'model energy': min_energy, 
        'model probability': dict_pred, 
        'model answer': model_ans, 
        'exact energies': dict_true, 
        'exact answer': exact_ans,
        'exact energy': dict_true[exact_ans],
        'circuit': qc
    }

In [5]:
# model, obj, args, num_qubit = mysetup('3to5')
df = pd.DataFrame(columns=['type', 'size', 'number of trial', 'temperature', 'seed', 'model answer', 'model energy', 'exact answer', 'exact energy'])

# for _type in ['only3', '3to5']:
for _type in ['3to5']:

    model, obj, args, num_qubit, size_list = mysetup(_type)

    # for size in size_list:
    for size in [4, 5]:
        for nt in [1, 2, 5, 10, 20]:
            for temp in [0.1, 0.5, 1.0, 2.0, 10.0, -1.0]:
        
                if temp < 0:
                    is_random = True
                else:
                    is_random = False
        
                for seed in tqdm(range(1000), desc=f'[type: {_type}, size: {size}, nt: {nt}, temp:{temp}]'):
                    fix_seed(seed)
                    adj, size, record = generate_data(args, num_clone=nt, seed=seed, device=model.device, size=size)
                    result = Mysolver(obj, model, adj, record, args, num_try=nt, temperature=temp, is_print=False, random_token=is_random)
        
                    result = {key: result[key] for key in result if key in df.columns}
                    result['type'] = _type
                    result['size'] = size
                    result['number of trial'] = nt
                    result['temperature'] = temp
                    result['seed'] = seed
        
                    df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)

            df.to_csv('./performance.csv')

[type: 3to5, size: 4, nt: 1, temp:0.1]:   0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/42757388.1.gpu/ipykernel_3958659/2376897661.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([result])], ignore_index=True)


[type: 3to5, size: 4, nt: 1, temp:0.5]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 4, nt: 1, temp:1.0]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 4, nt: 1, temp:2.0]:   0%|          | 0/1000 [00:00<?, ?it/s]

...

[type: 3to5, size: 5, nt: 20, temp:0.5]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 5, nt: 20, temp:1.0]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 5, nt: 20, temp:2.0]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 5, nt: 20, temp:10.0]:   0%|          | 0/1000 [00:00<?, ?it/s]

[type: 3to5, size: 5, nt: 20, temp:-1.0]:   0%|          | 0/1000 [00:00<?, ?it/s]